In [1]:
pip install dash==0.29.0

  Using cached dash-0.29.0-py3-none-any.whl
  Attempting uninstall: dash
    Found existing installation: dash 2.18.1
    Uninstalling dash-2.18.1:
      Successfully uninstalled dash-2.18.1
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dash-bootstrap-components 1.6.0 requires dash>=2.0.0, but you have dash 0.29.0 which is incompatible.


In [2]:
pip install --upgrade dash plotly

  Using cached dash-2.18.1-py3-none-any.whl.metadata (10 kB)
Using cached dash-2.18.1-py3-none-any.whl (7.5 MB)
  Attempting uninstall: dash
    Found existing installation: dash 0.29.0
    Uninstalling dash-0.29.0:
      Successfully uninstalled dash-0.29.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade dash dash-core-components dash-html-components dash-bootstrap-components

Note: you may need to restart the kernel to use updated packages.


In [15]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score

# Initialize Dash app
app = dash.Dash(__name__)

# Load the dataset
safaricom_engineered = pd.read_csv('safaricom_data_engineered.csv')

# Data preprocessing
X = safaricom_engineered.drop(columns=['Customer ID', 'Churn Indicator'])
y = safaricom_engineered['Churn Indicator']

# One-hot encoding categorical variables
X = pd.get_dummies(X, columns=['Region', 'Subscription Type', 'Payment Method'], drop_first=True)
X['Gender'] = X['Gender'].map({'Male': 1, 'Female': 0})

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model training
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# ROC Curve
y_prob = model.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_prob)
roc_auc = roc_auc_score(y_test, y_prob)

# Feature importance
importances = model.feature_importances_
feature_names = X.columns
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances}).sort_values(by='Importance', ascending=False)

# Plotly figures
# Churn distribution
churn_dist_fig = px.histogram(safaricom_engineered, x='Churn Indicator', title="Churn vs. Non-Churn Distribution")

# Confusion Matrix
conf_matrix_fig = go.Figure(data=go.Heatmap(
    z=conf_matrix,
    x=['No Churn', 'Churn'],
    y=['No Churn', 'Churn'],
    colorscale='Blues',
    text=conf_matrix,
    hoverinfo='text',
    showscale=False))

conf_matrix_fig.update_layout(title="Confusion Matrix", xaxis_title="Predicted", yaxis_title="Actual")

# ROC Curve
roc_curve_fig = go.Figure()
roc_curve_fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f'ROC Curve (AUC = {roc_auc:.2f})'))
roc_curve_fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random Classifier', line=dict(dash='dash')))
roc_curve_fig.update_layout(title="ROC Curve", xaxis_title="False Positive Rate", yaxis_title="True Positive Rate")

# Feature Importance
feature_importance_fig = px.bar(feature_importance_df, x='Importance', y='Feature', title="Feature Importance")

# Layout of the dashboard
app.layout = html.Div([
    html.H1("Safaricom Customer Churn Prediction Dashboard"),
    
    # Churn Distribution
    html.Div([
        dcc.Graph(figure=churn_dist_fig)
    ], style={'width': '48%', 'display': 'inline-block'}),

    # Confusion Matrix
    html.Div([
        dcc.Graph(figure=conf_matrix_fig)
    ], style={'width': '48%', 'display': 'inline-block'}),

    # ROC Curve
    html.Div([
        dcc.Graph(figure=roc_curve_fig)
    ], style={'width': '48%', 'display': 'inline-block'}),
    
    # Feature Importance
    html.Div([
        dcc.Graph(figure=feature_importance_fig)
    ], style={'width': '48%', 'display': 'inline-block'})
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
